In [1]:
!unzip /content/drive/MyDrive/224x224.zip -d /content/

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import numpy as np
import pandas as pd
from skimage.io import imread
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
names = ['id', 'png', 'url', 'title', 'author', 'target', 'genre']
train_dataframe = pd.read_csv('./book30-listing-train.csv', names = names, encoding='"ISO-8859-1"')
train_dataframe.head()

FileNotFoundError: [Errno 2] No such file or directory: './book30-listing-train.csv'

In [5]:
print(len(train_dataframe))
genres = [1, 2, 4, 5, 7, 15, 20, 23, 24, 27]
train_dataframe = train_dataframe[train_dataframe['target'].isin(genres)]
print(len(train_dataframe))
train_dataframe

51300
17100


,id,png,url,title,author,target,genre
0,1588345297,1588345297.jpg,http://ecx.images-amazon.com/images/I/51l6XIoa...,With Schwarzkopf: Life Lessons of The Bear,Gus Lee,1,Biographies & Memoirs
1,1404803335,1404803335.jpg,http://ecx.images-amazon.com/images/I/51UJnL3T...,"Magnets: Pulling Together, Pushing Apart (Amaz...",Natalie M. Rosinsky,4,Children's Books
7,73402656,0073402656.jpg,http://ecx.images-amazon.com/images/I/51WccSzF...,Chemistry: The Molecular Nature of Matter and ...,Martin Silberberg,23,Science & Math
11,137029039,0137029039.jpg,http://ecx.images-amazon.com/images/I/51T6dO1X...,Day Trading Options: Profiting from Price Dist...,Jeff Augen,2,Business & Money
12,806965339,0806965339.jpg,http://ecx.images-amazon.com/images/I/71SDZJBG...,"Breads, Cakes, Rolls & More from Your Bread Ma...",Rob Wanless,7,"Cookbooks, Food & Wine"
...,...,...,...,...,...,...,...
49482,1503949508,1503949508.jpg,http://ecx.images-amazon.com/images/I/51QE2xIh...,Cimarronin: The Complete Graphic Novel,Neal Stephenson,5,Comics & Graphic Novels
49483,1616556218,1616556218.jpg,http://ecx.images-amazon.com/images/I/61G6ddi3...,Plants vs Zombies: Timepocalypse,Paul Tobin,5,Comics & Graphic Novels
49484,1452149682,1452149682.jpg,http://ecx.images-amazon.com/images/I/51dImyTR...,Grumpy Cat: No-It-All: Everything You Need to No,Grumpy Cat,5,Comics & Graphic Novels
49485,1613281218,1613281218.jpg,http://ecx.images-amazon.com/images/I/5121j8kD...,101 Questions about the Bible and Christianity...,Art Ayris,5,Comics & Graphic Novels


In [ ]:
img_array = []
for png in train_dataframe['png']:
    #print(png)
    img_array.append(np.transpose(imread('/content/224x224/' + png), (2, 0, 1)).astype(np.float32) / 255.0)
img_array = np.array(img_array)

In [ ]:
targets_array = []
for target in train_dataframe['target']:
    targets_array.append([1.0 if target == x else 0.0 for x in genres])
targets_array = np.array(targets_array)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(img_array, targets_array, test_size=0.2, random_state=28, stratify = train_dataframe['target'])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(img_array, np.array(train_dataframe['target']).astype(np.float32), test_size=0.2, random_state=28, stratify = train_dataframe['target'])

In [ ]:
X_train = (torch.from_numpy(X_train)).to(device)
X_test = (torch.from_numpy(X_test)).to(device)
y_train = (torch.from_numpy(y_train)).to(device)
y_test = (torch.from_numpy(y_test)).to(device)

In [ ]:
train_dataset = utils.data.TensorDataset(X_train, y_train)
train_dataLoader = utils.data.DataLoader(train_dataset, batch_size = 16, shuffle = True)
test_dataLoader = utils.data.DataLoader(utils.data.TensorDataset(X_test,y_test), batch_size = 16, shuffle = True)

In [ ]:
'''
class BookCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # input 3x224x224, output 15x108x108
        self.conv1 = nn.Conv2d(3, 15, kernel_size=(9,9), stride=2)
        self.act1 = nn.ReLU()
        # input 15x108x108, output 15x36x36
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3))
        self.drop1 = nn.Dropout(0.2)

        # input 15x36x36, output 15x36x36
        self.conv2 = nn.Conv2d(15, 60, kernel_size=(5,5), padding = 2, stride=1)
        self.act2 = nn.ReLU()
        # input 15x36x36, output 60x18x18
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.drop2 = nn.Dropout(0.2)

        # input 60x18x18, output 60x18x18
        self.conv3 = nn.Conv2d(60, 120, kernel_size=(3,3), padding = 1, stride=1)
        self.act3 = nn.ReLU()
        # input 120x9x9, 120x9x9
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))

        self.flat = nn.Flatten()

        self.fc5 = nn.Linear(9720, 4800)
        self.act5 = nn.ReLU()
        self.drop5 = nn.Dropout(0.3)

        self.fc6 = nn.Linear(4800, 1600)
        self.act6 = nn.ReLU()
        self.drop6 = nn.Dropout(0.3)

        self.fc7 = nn.Linear(1600, 600)
        self.act7 = nn.ReLU()
        self.drop7 = nn.Dropout(0.3)

        self.fc8 = nn.Linear(600, 100)
        self.act8 = nn.ReLU()
        self.drop8 = nn.Dropout(0.3)

        self.fc9 = nn.Linear(100, 10)
        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool1(x)
        x = self.drop1(x)

        x = self.act2(self.conv2(x))
        x = self.pool2(x)
        x = self.drop2(x)

        x = self.act3(self.conv3(x))
        x = self.pool3(x)

        x = self.flat(x)

        x = self.act5(self.fc5(x))
        x = self.drop5(x)

        x = self.act6(self.fc6(x))
        x = self.drop6(x)

        x = self.act7(self.fc7(x))
        x = self.drop7(x)

        x = self.act8(self.fc8(x))
        x = self.drop8(x)

        x = self.fc9(x)
        #x = self.logSoftmax(self.fc9(x))

        return x
'''
class BookCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # input 3x224x224, output 96x54x54
        self.conv1 = nn.Conv2d(3, 96, kernel_size=(11,11), stride=4, padding=3)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 3), stride=2)

        self.conv2 = nn.Conv2d(96, 256, kernel_size=(5,5), stride=1, padding=2)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 3), stride=2)

        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3,3), stride=1, padding=1)
        self.act3 = nn.ReLU()
        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3,3), stride=1, padding=1)
        self.act4 = nn.ReLU()
        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3,3), stride=1, padding=1)
        self.act5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=(3, 3), stride=2)

        self.drop5 = nn.Dropout(0.5)

        self.flat = nn.Flatten()

        self.fc5 = nn.Linear(9216, 4096)
        self.act5 = nn.ReLU()
        self.drop5 = nn.Dropout(0.5)

        self.fc6 = nn.Linear(4096, 4096)
        self.act6 = nn.ReLU()

        self.fc7 = nn.Linear(4096, 1000)
        self.act7 = nn.ReLU()

        self.fc8 = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool1(x)

        x = self.act2(self.conv2(x))
        x = self.pool2(x)

        x = self.act3(self.conv3(x))

        x = self.act4(self.conv4(x))

        x = self.act5(self.conv5(x))
        x = self.pool5(x)

        x = self.drop5(x)

        x = self.flat(x)

        x = self.act5(self.fc5(x))
        x = self.drop5(x)

        x = self.act6(self.fc6(x))

        x = self.act7(self.fc7(x))

        x = self.fc8(x)

        return x

In [ ]:
model = BookCNN()
model.to(device)

In [ ]:
def discretize(contValue: torch.tensor, discValues: torch.tensor) -> torch.tensor:
    diff    = discValues - contValue
    absDiff = torch.abs(diff)
    minIdx  = torch.argmin(absDiff)
    dt      = contValue + diff[minIdx]
    return dt

In [ ]:
loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum = 0.9)

#genre_tensor = torch.from_numpy(np.array(genres) / 30.0).to(device)

n_epochs = 20
for epoch in range(n_epochs):
    avg_loss = 0.0
    count = 0
    for inputs, labels in train_dataLoader:
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        avg_loss += loss
        count += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(avg_loss / count)
    acc = 0
    count = 0
    for inputs, labels in test_dataLoader:
        y_pred = model(inputs)
        #acc += torch.isclose(discretize(y_pred, genre_tensor), (labels / 30.0)).float().sum()

        acc += (torch.argmax(y_pred, 1) == torch.argmax(labels, 1)).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %f%%" % (epoch, acc*100))

torch.save(model.state_dict(), "BookCNN.pth")